## Installing Elasticsearch

If your on mac and can brew install it:

` > brew install elasticsearch`

` > sudo ln -sfv /usr/local/opt/elasticsearch/*.plist ~/Library/LaunchAgents`

The default endpoint is `localhost:9200`.  We can test it like so:

` > curl localhost:9200`

`
{
  "name" : "Carolyn Trainer",
  "cluster_name" : "elasticsearch_yourname",
  "version" : {
    "number" : "2.0.0",
    "build_hash" : "de54438d6af8f9340d50c5c786151783ce7d6be5",
    "build_timestamp" : "2015-10-22T08:09:48Z",
    "build_snapshot" : false,
    "lucene_version" : "5.2.1"
  },
  "tagline" : "You Know, for Search"
}
`


Here are some other resources:

[1](https://github.com/sloanahrens/qbox-blog-code/blob/master/ch_1_local_ubuntu_es/install_es.sh) or [2](http://joelabrahamsson.com/elasticsearch-101/)

Otherwise, google is your friend.

## Inserting the Movielens dataset

A good portion of this proceedure, and some of the code, is taken from [here](https://www.mapr.com/products/mapr-sandbox-hadoop/tutorials/recommender-tutorial)

[Movielense dataset](http://grouplens.org/datasets/movielens/)

` > wget http://files.grouplens.org/datasets/movielens/ml-latest.zip `    

To create "movielense" mapping:

`
curl -XPUT 'http://localhost:9200/movielen' -d '
{
  "mappings": {
    "film" : {
      "properties" : {
        "numFields" : { "type" :   "integer" }
      }
    }
  }
}'
`

`{"acknowledged":true}%`

In `libraray_home/data/movielense` there is a script called "index.py".  This script with convert the movie.csv into a file of paired json docs suitable for inserting into elasticsearch.  The first doc creates the document, and the second specifies the fields.

Drop `index.py` into the folder with the unzipped movielens data and run:

` > python index.py > index.json `

Then, use curl to bulk insert the data into elasticsearch

` > curl -s -XPOST localhost:9200/_bulk --data-binary @index.json; echo`

There should be a bunch of output, the last element of which should look like:

` {"create":{"_index":"movielens","_type":"film","_id":"151711","_version":1,"_shards":{"total":2,"successful":1,"failed":0},"status":201}}]} `

## Querying Elasticsearch

To test out some queries in ES, we can use the chrome plugin Sense (beta).  
We can search for some dramas with a query like this:

`
GET _search
{
   "query": {
      "match": {
          "genre" : "drama"
      }
   },
   "size" : 8
}
`

We can also curl ES and search like so:

`
curl -XPOST "http://localhost:9200/_search" -d'
{
    "query": {
        "query_string": {
            "query": "kill"
        }
    }
}'
`

which should return:

`
{"took":14,"timed_out":false,"_shards":{"total":10,"successful":10,"failed":0},"hits":{"total":78,"max_score":3.0908446,"hits":[{"_index":"movielens","_type":"film","_id":"390","_score":3.0908446,"_source":{ "id": "390", "title" : "Faster Pussycat! Kill! Kill!", "year":"1965" , "genre":["Action", "Crime", "Drama"] }},{"_index":"movielens","_type":"film","_id":"94427","_score":3.0209367,"_source":{ "id": "94427", "title" : "Shadow Kill", "year":"2002" , "genre":["Drama"] }},{"_index":"movielens","_type":"film","_id":"132112","_score":2.782512,"_source":{ "id": "132112", "title" : "Good Kill", "year":"2014" , "genre":["Thriller"] }},{"_index":"movielens","_type":"film","_id":"4764","_score":2.6733258,"_source":{ "id": "4764", "title" : "Kill Me Later", "year":"2001" , "genre":["Romance", "Thriller"] }},{"_index":"movielens","_type":"film","_id":"86628","_score":2.6733258,"_source":{ "id": "86628", "title" : "Kill the Irishman", "year":"2011" , "genre":["Action", "Crime"] }},{"_index":"movielens","_type":"film","_id":"132958","_score":2.6733258,"_source":{ "id": "132958", "title" : "The Kill Team", "year":"2013" , "genre":["Documentary", "War"] }},{"_index":"movielens","_type":"film","_id":"61697","_score":2.6226687,"_source":{ "id": "61697", "title" : "Righteous Kill", "year":"2008" , "genre":["Crime", "Mystery", "Thriller"] }},{"_index":"movielens","_type":"film","_id":"70008","_score":2.6226687,"_source":{ "id": "70008", "title" : "Kill Your Darlings", "year":"2006" , "genre":["Comedy", "Drama"] }},{"_index":"movielens","_type":"film","_id":"86677","_score":2.6226687,"_source":{ "id": "86677", "title" : "Kill Theory", "year":"2009" , "genre":["Horror", "Thriller"] }},{"_index":"movielens","_type":"film","_id":"101428","_score":2.6226687,"_source":{ "id": "101428", "title" : "Kill for Me", "year":"2013" , "genre":["Drama", "Thriller"] }}]}}%
`

More details about the query DSL can be found [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/index.html)